In [1]:
import numpy as np
import math as m
import urllib
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pandas as pd
import sklearn
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

def divide(X_train,Y_train):
#     F_and_L = zip(X_train, Y_train)
    X_train=[X_train[i][0] for i in range(len(X_train))]
    d={"x":pd.Series(X_train),"Label":pd.Series(Y_train)}
    F_L = pd.DataFrame(d)
    setosa=pd.DataFrame(columns=('x', 'Label'))
    versicolor=pd.DataFrame(columns=('x', 'Label'))
    for i in range(len(F_L)):
        if(F_L.Label[i] == 1 ):
            setosa.loc[i]=[F_L.x[i],F_L.Label[i]]
        else:
            versicolor.loc[i]=[F_L.x[i],F_L.Label[i]]
#     print len(setosa.x)      
    return setosa, versicolor
    
def member(setosa, versicolor,X_train,Y_train):
    setosa_mean=np.mean(setosa.x)
    versi_mean=np.mean(versicolor.x)
    setosa_var=np.var(setosa.x)
    versi_var=np.var(versicolor.x)
#     print setosa_mean, setosa_var, versi_mean, versi_var
    g1=[]
    g2=[]
    d=pd.DataFrame(columns=('d', 'Label'))
    for i in range(len(X_train)):
        g1x= -m.log(m.sqrt(setosa_var))- (((X_train[i] - setosa_mean) ** 2) / (2 *(setosa_var )))+ m.log(0.5)
#         g1x=-m.log(setosa_var) - 0.5 * ((float(X_train[i] ) - float(setosa_mean)) ** 2 / setosa_var ** 2) + m.log(0.5)
        g1.append(g1x)
        g2x= -m.log(m.sqrt(versi_var))- (((X_train[i] - versi_mean) ** 2) / (2*(versi_var))) + m.log(0.5)
#         g2x=-m.log(versi_var) - 0.5 * ((float(X_train[i] ) - float(versi_mean)) ** 2 / versi_var ** 2) + m.log(0.5)
        g2.append(g2x)
#         discriminant= g1[i]-g2[i]
#         d.append(discriminant)
        d.loc[i]=[g1[i]-g2[i], Y_train[i]]
#     print g1
#     print g2
    return d
#     print d

def classify(d,Fun):
    c1=pd.DataFrame(columns=('c1','ALabel', 'PLabel','FunLabel'))
    c2=pd.DataFrame(columns=('c2', 'ALabel','PLabel','FunLabel'))
    for i in range(len(d)):
        if(d.d[i]>=0):
            c1.loc[i]=[d.d[i],d.Label[i],1,Fun[i]]
        else:
            c2.loc[i]=[d.d[i],d.Label[i],2,Fun[i]]
#     print c1,c2
    c=pd.concat([c1,c2])
#     print c
    return c
    
def confusion_mat(c):
#     df.colName.value_counts()
    mse=mean_squared_error(c.ALabel, c.PLabel)
    print "Mean Squared Error: ", mse
    TP=0
    FP=0
    TN=0
    FN=0
    for i in range(len(c)):
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==1):
            TP+=1
        if (c.ALabel[i]== c.PLabel[i] and c.ALabel[i]==2):
            TN+=1
        if(c.ALabel[i]==2 and c.PLabel[i]==1):
            FP+=1
        if(c.ALabel[i]==1 and c.PLabel[i]==2):
            FN+=1
    con_mat=np.matrix([[TP,FP],[FN,TN]])
    Accuracy=float((TP+TN))/float((TP+FP+TN+FN))
    print "Accuracy: ",Accuracy
    return con_mat

    
def k_fold(X, Y):
#     fold_size = len(X) / k_fold  
    with_class={}
    final={}
    kf=sklearn.cross_validation.KFold(n=len(X), n_folds=10, shuffle=False,random_state=None)
    for train_index, test_index in kf:
        X_train,X_test=X[train_index], X[test_index]
        Y_train,Y_test=Y[train_index], Y[test_index]
        setosa, versicolor= divide(X_train,Y_train) 
#             calculate_mean_variance(X_train,Y_train) 
#             print setosa_mean, versi_mean, setosa_var, versi_var
        d=member(setosa, versicolor,X_train,Y_train)
#             print np.matrix(X_train)
        lda = LinearDiscriminantAnalysis()
        clf = lda.fit(X_train,Y_train)
        Fun1=clf.predict(X_train)
        c=classify(d,Fun1)
        con_mat=confusion_mat(c)
        p1= (float(con_mat[0,0])/float((con_mat[0,0]+con_mat[0,1])))
        r1=(float(con_mat[0,0])/float((con_mat[0,0]+con_mat[1,0])))
        p2= (float(con_mat[1,1])/float((con_mat[1,1]+con_mat[1,0])))
        r2=(float(con_mat[1,1])/float(con_mat[1,1]+con_mat[0,1]))
        precision=[p1,p2]
        recall=[r1,r2]
        print "Confusion Matrix: ",con_mat
        print "Precision: ",precision
        print "Recall: " ,recall
        F1=2*((p1*r1)/(p1+r1))
        F2=2*((p2*r2)/(p2+r2))
        print "F_measure: ",[F1,F2]


def main():
    iris = load_iris()
    X1 = iris.data
    y1 = iris.target
#     X=iris.data[:,0]
    X = iris.data[:,0:1] 
    Y = iris.target
    enc = LabelEncoder()
    label_encoder = enc.fit(Y)
    y = label_encoder.transform(Y) + 1
    x=X[0:100]
    y=y[0:100]
    x1=[x[i][0] for i in range(len(x))]
    lda = LinearDiscriminantAnalysis()
    clf = lda.fit(x,y)
    Fun=clf.predict(x)
#     without fold
    print("Performance with 75% training size")
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
    setosa, versicolor= divide(X_train,y_train) 
    d=member(setosa, versicolor,X_test,y_test)
    c=classify(d,Fun)
    con_mat=confusion_mat(c)
    p1= (float(con_mat[0,0])/float((con_mat[0,0]+con_mat[0,1])))
    r1=(float(con_mat[0,0])/float((con_mat[0,0]+con_mat[1,0])))
    p2=(float(con_mat[1,1])/float((con_mat[1,1]+con_mat[1,0])))
    r2=(float(con_mat[1,1])/float(con_mat[1,1]+con_mat[0,1]))
    precision=[p1,p2]
    recall=[r1,r2]
    print "Confusion Matrix: ",con_mat
    print "Precision: ",precision
    print "Recall: " ,recall
    F1=2*((p1*r1)/(p1+r1))
    F2=2*((p2*r2)/(p2+r2))
    print "F_measure: ",[F1,F2]
#     with fold
    print("******")
    print("Performance with 10 fold")
    k_fold(x,y)
    

    
if __name__ == "__main__":main()

Performance with 75% training size
Mean Squared Error:  0.08
Accuracy:  0.92
Confusion Matrix:  [[12  0]
 [ 2 11]]
Precision:  [1.0, 0.8461538461538461]
Recall:  [0.8571428571428571, 1.0]
F_measure:  [0.923076923076923, 0.9166666666666666]
******
Performance with 10 fold
Mean Squared Error:  0.122222222222
Accuracy:  0.877777777778
Confusion Matrix:  [[35  6]
 [ 5 44]]
Precision:  [0.8536585365853658, 0.8979591836734694]
Recall:  [0.875, 0.88]
F_measure:  [0.8641975308641976, 0.888888888888889]
Mean Squared Error:  0.111111111111
Accuracy:  0.888888888889
Confusion Matrix:  [[35  5]
 [ 5 45]]
Precision:  [0.875, 0.9]
Recall:  [0.875, 0.9]
F_measure:  [0.875, 0.9]
Mean Squared Error:  0.122222222222
Accuracy:  0.877777777778
Confusion Matrix:  [[35  6]
 [ 5 44]]
Precision:  [0.8536585365853658, 0.8979591836734694]
Recall:  [0.875, 0.88]
F_measure:  [0.8641975308641976, 0.888888888888889]
Mean Squared Error:  0.1
Accuracy:  0.9
Confusion Matrix:  [[37  6]
 [ 3 44]]
Precision:  [0.8604651